In [36]:
import cv2
from PIL import Image
import numpy as np

def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image

part_image = preprocess_image("data/dummy/part_1/part_1.png")
gripper_image = preprocess_image("data/dummy/part_1/gripper_2.png")

# gripper_image change black to white
# gripper_image[gripper_image == 0] = 255
# gripper_image[gripper_image == 255]

# convert to PIL image
# part_image = Image.fromarray(part_image)
# gripper_image = Image.fromarray(gripper_image)

# part_image.show()
# gripper_image.show()

In [37]:
def generate_edge_mask(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

part_edges = generate_edge_mask(part_image)


In [38]:
def is_valid_position(part_edges, gripper_image, x, y):
    part_height, part_width = part_edges.shape
    gripper_height, gripper_width = gripper_image.shape
    
    if x + gripper_width > part_width or y + gripper_height > part_height:
        return False
    
    for i in range(gripper_height):
        for j in range(gripper_width):
            if gripper_image[i, j] > 0 and part_edges[y + i, x + j] > 0:
                return False
    return True

def find_valid_position(part_edges, gripper_image):
    part_height, part_width = part_edges.shape
    gripper_height, gripper_width = gripper_image.shape
    
    for y in range(part_height - gripper_height):
        for x in range(part_width - gripper_width):
            if is_valid_position(part_edges, gripper_image, x, y):
                return x, y
    return None

position = find_valid_position(part_edges, gripper_image)

In [39]:
position

(51, 9)

In [40]:
def place_gripper(part_image, gripper_image, position):
    x, y = position
    part_image_with_gripper = part_image.copy()
    part_image_with_gripper[y:y+gripper_image.shape[0], x:x+gripper_image.shape[1]] = gripper_image
    return part_image_with_gripper

if position:
    result_image = place_gripper(part_image, gripper_image, position)
    cv2.imwrite("data/Output_Images/test.png", result_image)
else:
    print("No valid position found for the gripper.")